# Домашнее задание «Основы парсинга и работы с API»

In [1]:
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup

<div class="alert alert-success">
<h2> Задание 1 </h2>
</div>

<div class="alert alert-block alert-info">
    <h3>Обязательная часть</h3>
</div>

Будем парсить страницу со свежеми новостям на https://habr.com/ru/all/.  
Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово.  
Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).  
В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [2]:
KEYWORDS = ['python', 'парсинг']

In [3]:
def get_fresh_habr_posts_by_preview_keywords(keywords):
    habr_posts_df = pd.DataFrame()
    posts = (BeautifulSoup(requests.get('https://habr.com/ru/all/').text, 'html.parser')
             .find_all('article'))
    for post in posts:
        for word in keywords:
            if word in post.text.lower():
                post_title = post.find('a', class_='post__title_link').text
                post_link = post.find('a', class_='post__title_link').attrs.get('href')              
                post_date = (BeautifulSoup(requests.get(post_link).text, 'html.parser')
                        .find('meta', property='aiturec:datetime').attrs.get('content'))           
                row = {'date': post_date, 'title': post_title, 'link': post_link}
                habr_posts_df = pd.concat([habr_posts_df, pd.DataFrame([row])]) 
                break
    return habr_posts_df[['date', 'title', 'link']].reset_index(drop = True)

In [4]:
get_fresh_habr_posts_by_preview_keywords(KEYWORDS)

,date,title,link
0,2021-02-13T21:31:19+03:00,7 полезных расширений VS Code для Python-разра...,https://habr.com/ru/company/timeweb/blog/542286/


<div class="alert alert-block alert-info">
    <h3>Дополнительная часть</h3>
</div>

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [5]:
def get_fresh_habr_posts_by_keywords(keywords):
    habr_posts_df = pd.DataFrame()
    posts = (BeautifulSoup(requests.get('https://habr.com/ru/all/').text, 'html.parser')
             .find_all('article'))
    for post in posts:
        post_link = post.find('a', class_='post__title_link').attrs.get('href')
        post_html = BeautifulSoup(requests.get(post_link).text, 'html.parser')
        for word in keywords:
            if word in post_html.find('article', class_='post post_full').text.lower():
                post_title = post.find('a', class_='post__title_link').text            
                post_date = post_html.find('meta', property='aiturec:datetime').attrs.get('content') 
                post_text = post_html.find('div', id='post-content-body').text
                row = {'date': post_date, 'title': post_title, 'link': post_link, 'text': post_text}
                habr_posts_df = pd.concat([habr_posts_df, pd.DataFrame([row])]) 
                break
    return habr_posts_df[['date', 'title', 'link', 'text']].reset_index(drop = True)

In [6]:
get_fresh_habr_posts_by_keywords(KEYWORDS)

,date,title,link,text
0,2021-02-13T22:11:09+03:00,Инструменты для участников соревнований по маш...,https://habr.com/ru/post/542292/,Соревнования по машинному обучению — относител...
1,2021-02-13T21:31:19+03:00,7 полезных расширений VS Code для Python-разра...,https://habr.com/ru/company/timeweb/blog/542286/,Наиболее важные и полезные расширения VS Code ...


<div class="alert alert-success">
<h2> Задание 2 </h2>
</div>

<div class="alert alert-block alert-info">
    <h3>Обязательная часть</h3>
</div>

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса https://www.avast.com/hackcheck/.  
Список email-ов задаем переменной в начале кода:  

EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [109]:
EMAIL = ['xxx@x.ru', 'yyy@y.com', 'kochetkov.vitt@gmail.com'] # leaks: 8, 19, 0

In [110]:
def check_emails_hacks_by_avast(emails):
    emails_hacks_df = pd.DataFrame()
    URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
    headers = {'Vaar-Version': '0', 'Vaar-Header-App-Product': 'hackcheck-web-avast'}
    data = requests.post(URL, json={'emailAddresses': emails}, headers=headers).json()
    
    for email, value in data['summary'].items():
        for breach in value['breaches']:
            leak_date = data['breaches'][str(breach)]['publishDate']
            leak_source = data['breaches'][str(breach)]['site']
            description = data['breaches'][str(breach)]['description']
            row = {'email': email, 'leak_date': leak_date, 
                   'leak_source': leak_source, 'description': description}
            emails_hacks_df = pd.concat([emails_hacks_df, pd.DataFrame([row])]) 
            
    return emails_hacks_df[['email', 'leak_date', 'leak_source', 'description']].reset_index(drop = True)

In [111]:
check_emails_hacks_by_avast(EMAIL)

,email,leak_date,leak_source,description
0,xxx@x.ru,2019-03-28T00:00:00Z,verifications.io,Big data e-mail verification platform verifica...
1,xxx@x.ru,2020-05-21T00:00:00Z,vk.com,"At some time in 2020, the Russian social netwo..."
2,xxx@x.ru,2017-02-14T00:00:00Z,parapa.mail.ru,"In July and August 2016, two criminals execute..."
3,xxx@x.ru,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
4,xxx@x.ru,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
5,xxx@x.ru,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
6,xxx@x.ru,2017-01-31T00:00:00Z,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
7,xxx@x.ru,2016-10-23T00:00:00Z,imesh.com,"In June 2016, a cache of over 51 million user ..."
8,yyy@y.com,2019-03-28T00:00:00Z,verifications.io,Big data e-mail verification platform verifica...
9,yyy@y.com,2019-02-21T00:00:00Z,www.dangdang.com,"This is a list of email addresses only, and as..."


<div class="alert alert-block alert-info"><h3>Дополнительная часть</h3></div>

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods , вам поможет метод https://vk.com/dev/wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

In [112]:
# coming soon